In [1]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib

import vis.fields
import matplotlib.cm as cm
import time

In [2]:
# Set to True in the options below if you already optimized a specific linkage
# and you would like to reuse the optimized linkage. Loading is performed in
# place of the full design optimization.
# NOTE: Doesn't seem to work at the moment, leave at False
LOAD_OPTIMIZED_DOFS = False

# Set to False if strips cannot be labeled into families for a model
USE_FAMILY_LABEL = False

In [3]:
# Regular Torus 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1/200, 1/20]
MODEL_NAME = "regular_torus_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "{}_strip".format(MODEL_NAME)

In [4]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, InterleavingType.triaxialWeave)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

In [5]:
def initialize_normal_linkage(cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    return l

In [6]:
def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

In [7]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

### Generate straight linkage equilibrium for testing later

In [8]:
# straight_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)

In [9]:
# view = linkage_vis.LinkageViewer(straight_linkage, width=1024, height=640)
# view.setCameraParams(default_camera_parameters)
# view.show()

In [10]:
# straight_linkage.attraction_weight = 0
# with so(): straight_linkage, initial_view = get_linkage_eqm(straight_linkage, cam_param = default_camera_parameters, opt = OPTS)
# initial_view.show()

In [11]:
# straight_rod_dof = straight_linkage.getDoFs()

In [12]:
# straight_linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Elastic)

### Create Surface Attracted Linkage and solve for design parameters with fixed joint positions

In [13]:
with so(): linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
# with so(): linkage = initialize_normal_linkage(model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
save_tgt_joint_pos = linkage.jointPositions()

In [28]:
joint_normals = np.array([linkage.joint(i).normal for i in range(linkage.numJoints())])
joint_projection_normals = linkage.get_closest_point_normal(linkage.jointPositions())
joint_projection_normals = joint_projection_normals.reshape(joint_normals.shape)
for i in range(len(joint_normals)):
    print(np.dot(joint_projection_normals[i], joint_normals[i]))

In [21]:
view = linkage_vis.LinkageViewer(linkage)
view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [29]:
loadedDoFs = np.load("dof_files/{}.npy".format(MODEL_NAME))
linkage.setExtendedDoFs(loadedDoFs)

In [30]:
view.update()

In [16]:
rw = 0.1
sw = 0.1

In [17]:
iterateData = []
iterateSegmentRL = []
iterateEdgeRL = []
!mkdir -p dps_iterates
!rm -f dps_iterates/*.msh dps_iterates/*.png
prev_vars = linkage.getExtendedDoFsPSRL()
def callback(prob, i):
    global prev_vars
    curr_vars = linkage.getExtendedDoFsPSRL()
    linkage.saveVisualizationGeometry(f'dps_iterates/it_{i}.msh', averagedMaterialFrames=True, averagedCrossSections=True)
    iterateSegmentRL.append(linkage.getPerSegmentRestLength())
    iterateEdgeRL.append(np.array([s.rod.restLengths() for s in linkage.segments()]).ravel())
    iterateData.append([prob.restKappaSmoothness(),
                        linkage.totalRestLength(),
                        linkage.designParameterSolve_energy(),
                        np.linalg.norm(curr_vars - prev_vars)])
    prev_vars[:] = curr_vars
    if (i % 20) != 0: return
    view.update()

In [18]:
dpo = elastic_rods.get_designParameter_optimizer(linkage, rw, sw, callback=callback)
dpo.options.niter = 10000
dpp = dpo.get_problem()

In [19]:
cr = dpo.optimize()

0	29.0179	2573.87	2573.87	1	1
1	14.4416	884.6	884.6	1	1
2	9.33146	296.416	296.416	1	1
3	6.95406	117.746	117.746	1	1
4	5.46968	61.3448	61.3448	1	1
5	4.29073	42.8066	42.8066	1	1
6	3.30289	36.2496	36.2496	1	1
7	2.55492	33.7101	33.7101	1	1
8	2.04101	44.1353	44.1353	1	1
9	1.70963	39.5592	39.5592	1	1
10	1.50205	42.7576	42.7576	1	1
11	1.43726	14.0603	14.0603	1	1
12	1.37954	15.5924	15.5924	1	1
13	1.32967	15.5095	15.5095	1	1
14	1.28687	16.0743	16.0743	1	1
15	1.24955	20.495	20.495	1	1
16	1.22791	15.7815	15.7815	1	1
17	1.20448	18.8155	18.8155	1	1
18	1.19808	15.6141	15.6141	1	1
19	1.19179	15.6147	15.6147	1	1
20	1.17996	15.9044	15.9044	1	1
21	1.16067	17.1781	17.1781	0.5	1
22	1.147	19.2735	19.2735	0.25	1
23	1.14384	20.8586	20.8586	1	1
24	1.13598	17.2867	17.2867	1	1
25	1.13252	16.3528	16.3528	1	1
26	1.12833	16.3473	16.3473	1	1
27	1.12163	16.4834	16.4834	1	1
28	1.11314	16.8708	16.8708	1	1
29	1.10627	17.6725	17.6725	1	1
30	1.1036	17.1201	17.1201	1	1
31	1.10179	16.8844	16.8844	1	1
32	1.09921	16.8847	16.

In [31]:
import linkage_optimization

In [32]:
OPTS.niter = 200
linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
useCenterline = True
optimizer = linkage_optimization.WeavingOptimization(linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
optimizer.set_target_joint_position(save_tgt_joint_pos)
view.update()

In [33]:
optimizer.rl_regularization_weight = 1
optimizer.smoothing_weight = 10
optimizer.beta = 500000.0
optimizer.gamma = 1
algorithm = linkage_optimization.WeavingOptAlgorithm.NEWTON_CG
def update_viewer():
    view.update()

In [34]:
if not LOAD_OPTIMIZED_DOFS:
    optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)
else:
    loadedDoFs = np.load("dof_files/{}.npy".format(MODEL_NAME))
    linkage.setExtendedDoFs(loadedDoFs)
    view.update()

In [35]:
optimizer.setLinkageAttractionWeight(1e-5)
optimizer.set_holdClosestPointsFixed(False)

In [36]:
optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)

In [ ]:
## If you want to solve again after loading optimized DOFs (e.g. if the code changed)
# algorithm = elastic_rods.WeavingOptAlgorithm.NEWTON_CG
# optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)

In [ ]:
# Store DoFs to file
if not os.path.exists("dof_files"):
    os.makedirs("dof_files")
rawDoFs = linkage.getExtendedDoFs()
np.save("dof_files/{}.npy".format(MODEL_NAME), rawDoFs)

In [ ]:
linkage.energy()

In [ ]:
start_time = time.time()
with so(): validation_linkage, validation_view = get_linkage_eqm(optimizer.get_linesearch_weaver(), cam_param = default_camera_parameters, opt = OPTS, target_surf=INPUT_SURFACE_PATH)
print('compute equilibrium takes: ', time.time() - start_time)

In [ ]:
validation_linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Elastic)

In [ ]:
import structural_analysis
structural_analysis.weavingCrossingAnalysis(linkage)
v = structural_analysis.crossingForceFieldVisualization(linkage)
v.show()